In [ ]:
import numpy as np
from transformers import BertTokenizer, AutoModel
import torch
import datasets
import os

TOKENIZER_CACHE_DIR = 'cache/bert-tokenizer'
MODEL_CACHE_DIR = 'cache/pretrained_model'

In [18]:
BertTokenizer.from_pretrained('uer/gpt2-chinese-cluecorpussmall').save_pretrained(TOKENIZER_CACHE_DIR)

('cache/bert-tokenizer/tokenizer_config.json',
 'cache/bert-tokenizer/special_tokens_map.json',
 'cache/bert-tokenizer/vocab.txt',
 'cache/bert-tokenizer/added_tokens.json')

In [22]:
AutoModel.from_pretrained('uer/gpt2-chinese-cluecorpussmall').save_pretrained(MODEL_CACHE_DIR)